<b>Definitions</b>

Define the used TANGO devices and constants.
Change list of device proxies and "devices" and "tiles" to actually present and used tiles

Put all devices Online. Tiles must be in Maintenance mode in order to use the test signal generator.

In [1]:
import tango
import time
import json
import numpy as np

from ska_control_model import (
    AdminMode,
    CommunicationStatus,
    HealthState,
    PowerState,
    ResultCode,
    SimulationMode,
    TestMode,
)
# for time conversion
from datetime import datetime,timezone
RFC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

# define devices
station = tango.DeviceProxy('low-mccs/station/001')
subrack = tango.DeviceProxy('low-mccs/subrack/0001')
t1 = tango.DeviceProxy('low-mccs/tile/0001')
#t2 = tango.DeviceProxy('low-mccs/tile/0002')
#t3 = tango.DeviceProxy('low-mccs/tile/0003')
#t4 = tango.DeviceProxy('low-mccs/tile/0004')
#t5 = tango.DeviceProxy('low-mccs/tile/0005')
#t6 = tango.DeviceProxy('low-mccs/tile/0006')
#t7 = tango.DeviceProxy('low-mccs/tile/0007')
#t8 = tango.DeviceProxy('low-mccs/tile/0008')

#station.logginglevel=5
devices = [station, subrack, t1,] # t2, t3, t4, t5, t6, t7, t8]
tiles = [t1, ] # t4]

# Put everything online
for d in devices:
    d.adminmode = AdminMode.ONLINE
time.sleep(0.2)
# 
# Tiles must be in Maintenance mode to allow test signal generator
for t in tiles:
    t.adminMode = AdminMode.MAINTENANCE

<b> Local parameters </b>
<ul><li> csp_ingest_ip: IP address of the CSP ingest port</li>
    <li> lmc_ip: IP address of the LMC DAQ system</li>
    <li> input ADC: ADC input channel (0-31, 0-1 for antenna 1, 30-31 for antenna 16)</li>
    <li> csp_rounding: Depending on actual signal level, adjust signal level at channelizer output. Nominal value (4)  is appropriate for a sinewave with RMS input amplitude, as measured by adcPower, in the range 11 to 22. Value is ceil(log2(adcPower/1.4))
    <li> input_frequency: frequency of the input signal. Used to compute the beamformed channel</li></ul>
The beamformer beamforms 8 channels starting at the first even channel equal or lower to this one

In [10]:
csp_ingest_ip = "10.0.0.99"
lmc_ip = "10.0.0.99"
input_adcs = [9, 11, 12, 15]
input_frequency = 230e6  # use actual tone frequency
nof_channels = 8
csp_rounding = 4    # adequate for -2:+5 dBm, adcLevel=11:22
start_channel = int(round(input_frequency/800e6*1024))
if start_channel % 2 == 0:
    print(f"Signal is on beamformed channel 0, corresponding to TPM channel {start_channel}")
else: 
    print(f"Signal is on beamformed channel 1, corresponding to TPM channel {start_channel}")
    start_channel = start_channel -1

Signal is on beamformed channel 0, corresponding to TPM channel 294


<b>Turn TPM on</b>
<ul>
    <li>Turn TPM on if not already on.</li>
    <li>Wait for intialization if not already initialised</li> 
    <li>If initialisation succeeds <ul>
        <li>perform initial setup</li>
        <li>start the acquisition.</li></ul>
    <li>Set destination IP addresses</li>
    <li>At the end, print signal level on input ADCs</li></ul>

In [3]:
t1.on()
t = 0
while not t1.tileprogrammingstate in ['Initialised', 'Synchronised']:
    print(f"{t}: {t1.tileprogrammingstate}")
    time.sleep(2)
    t = t + 2
    if t > 60:
        break
if t > 60:
    print("Initialisation failed")
elif t1.tileprogrammingstate == 'Initialised':
    print(f"{t}: {t1.tileprogrammingstate}")
    t1.ConfigureStationBeamformer(json.dumps({
        "start_channel": 192,
        "n_channels": 8,
        "is_first": True,
        "is_last": True,
    }))
    start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+2), RFC_FORMAT)
    t1.StartAcquisition(json.dumps({"start_time": start_time}))
    time.sleep(3)
if t1.tileprogrammingstate == "Synchronised":
    t1.statictimedelays=np.zeros([32],dtype=int)
    t1.channeliserRounding=[csp_rounding]*512
    t1.cspRounding=[0]*384
    t1.SetLmcDownload(json.dumps({"destination_ip": lmc_ip, "mode": "10g"}))
    t1.SetLmcIntegratedDownload(json.dumps({"destination_ip": lmc_ip, "mode": "10g"}))
    t1.Configure40GCore(json.dumps({"core_id": 0, "arp_table_entry": 0, "destination_ip": csp_ingest_ip}))
    t1.Configure40GCore(json.dumps({"core_id": 1, "arp_table_entry": 0, "destination_ip": csp_ingest_ip}))
print(f"{t1.fpgaframetime}: Tile is in state {t1.tileprogrammingstate}")
time.sleep(0.5) # to allow for the total power detector to compute the total power
print(f"Input levels: {t1.adcPower}")

0: Off
2: Off
4: Off
6: Off
8: Off
10: Off
12: Off
14: NotProgrammed
16: NotProgrammed
18: NotProgrammed
20: NotProgrammed
22: Programmed
24: Programmed
26: Programmed
28: Programmed
30: Programmed
32: Programmed
34: Programmed
36: Programmed
38: Programmed
40: Initialised
2023-05-26T15:35:24.716111Z: Tile is in state Synchronised
Input levels: [0.7990456  1.000002   0.94524125 0.999986   0.9965921  0.99755093
 0.99282002 0.999998   0.0411831  1.75519522 0.89105232 1.78618066
 0.6730384  1.49351614 0.99675264 1.53159057 0.99645562 0.99981398
 0.47839566 0.97064225 0.9961023  0.999994   0.99767924 0.99055715
 0.51574359 1.49522393 0.87943485 1.51004101 0.9986891  1.54621698
 0.75236152 1.57578346]


Program te test generator to produce null samples except for selected input.
Input ADC can be identified from adcPower attribute above.
Then start the generator.

In [11]:
t1.StopBeamformer()
beamformer_table = []  # Definition of beamformer regions. Flat array
beam_channels = []      # logical channels in each region. 2d array
beam = 0
logical_channel = 0
for _ in input_adcs:
    beamformer_table += [start_channel,nof_channels,beam,1,0,0,0,0]
    beam_channels += [[logical_channel, (logical_channel+nof_channels)]]
    logical_channel += nof_channels
    beam += 1

# print(beamformer_table)
t1.SetBeamformerRegions(beamformer_table)

# Program beam calibration matrix to send single signal to each beam
cal_gain = 4.0  # we are using just one antenna, raise the gain
cal_matrix_even = [cal_gain, 0.0, 0.0, 0.0, cal_gain, 0.0, 0.0, 0.0]  # send Xpol to both
cal_matrix_odd  = [0.0, 0.0, cal_gain, 0.0, 0.0, 0.0, cal_gain, 0.0]  # send Ypol to both

for antenna in range(16):
    cal_coefs = [antenna*1.0] + [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] * 384
    beam = 0
    for adc in input_adcs:
        if adc // 2 == antenna:
            if adc & 1 == 0:
                antenna_coefs = cal_matrix_even
            else:
                antenna_coefs = cal_matrix_odd
            
            for logical_channel in range(beam_channels[beam][0], beam_channels[beam][1]):
                cal_element = logical_channel*8+1
                cal_coefs[cal_element:cal_element+8] = antenna_coefs
            # print(cal_coefs)
        t1.LoadCalibrationCoefficients(cal_coefs)
        beam += 1
        
t1.ApplyCalibration("")

print(t1.fpgaframetime)
start_time = datetime.strftime(datetime.fromtimestamp(time.time()+2), RFC_FORMAT)
t1.StartBeamformer(json.dumps({"start_time": start_time}))
time.sleep(2)
print(f"Beamformer running: {t1.isBeamformerRunning}")
current_rounding = t1.channeliserRounding[0] & 0x7
level = t1.adcPower[input_adcs[0]]
chan_level = level * 1.45 * 2**current_rounding
if chan_level > 110 or chan_level < 50:
    rounding = int(np.ceil(np.log2(level/1.4)))
    if rounding <0:
        rounding = 0
    if rounding > 7: 
        rounding = 7
    t1.channeliserRounding = [rounding]*512
    print(f"Adjust channeliser rounding to {rounding}")
else:
    print(f"Keeping old channeliser rounding {current_rounding}")
print(f"Input level: {t1.adcPower[input_adcs[0]]}")

2023-05-26T15:50:28.268511Z
Beamformer running: True
Adjust channeliser rounding to 3
Input level: 7.050497845298321


In [12]:
print(t1.adcpower)

[0.77994425 0.999998   0.88772409 0.999926   0.99286031 0.9986851
 0.94729991 0.99993    0.02513996 7.04941866 0.92370119 6.92737812
 0.28828168 5.60060981 0.93826257 5.64296195 0.98580485 0.99885331
 0.53455028 0.97865357 0.91879031 0.99974596 0.99289657 0.99335574
 0.21320711 5.60968316 0.56812766 5.67268658 0.98580485 5.60800576
 0.63816252 5.6482193 ]


In [13]:
for sample_type in ["raw", "channel"]: #, "beam"]:
    print(f"Sending {sample_type} samples to DAQ")
    t1.senddatasamples(json.dumps({"data_type": sample_type}))
    time.sleep(3)

Sending raw samples to DAQ
Sending channel samples to DAQ


In [10]:
t1.off()

API_CorbaException: TRANSIENT CORBA system exception: TRANSIENT_CallTimedout
(For more detailed information type: tango_error)
